In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

In [2]:
def get_L(n_array):
    L = n_array.sum()
    return L

def get_K(b_array):
    K = b_array.sum()
    return K

def get_u(c, p):
    sigma = p['sigma']
    u = (c**(1-sigma) -1)/(1-sigma)
    return u

def u_prime(c, p):
    sigma = p['sigma']
    u_prime = c**(-sigma)
    return u_prime
    
    return u_prime
def get_w(K, L, p):
    A = p['A']
    alpha = p['alpha']
    w = (1-alpha)*A*(K/L)**alpha
    return w

def get_r(K, L, p):
    alpha = p['alpha']
    A = p ['A']
    delta = p['delta']
    r = alpha*A*(L/K)**(1-alpha) - delta
    
    return r


def get_c(b, p, f):
    n = p['n']
    w = f['w'](f['K'](b), f['L'](n), p)
    r = f['r'](f['K'](b), f['L'](n), p)
    
    c = np.zeros(3)
    
    c[0] = w*p['n'][0] - b[0]
    c[1] = w*p['n'][1] + (1+r)*b[0]-b[1]
    c[2] = w*p['n'][2] + (1+r)*b[1]
    
    return c



In [3]:

def equations_ss_b(b, p, f):
    """ Takes in a guess for b and a dictionary of functions 
    f and a dictionary of elemtns p."""
    n = p['n']
    w = f['w'](f['K'](b), f['L'](n), p)
    r = f['r'](f['K'](b), f['L'](n), p)
    
    e1_l = f['u_p'](w-b[0], p)
    e1_r = p['beta']*(1+r)*f['u_p'](w+(1+r)*b[0]-b[1], p)
    
    e2_l = f['u_p'](w+(1+r)*b[0]-b[1], p)
    e2_r = p['beta']*(1+r)*f['u_p']((1+r)*b[1]+0.2*w, p)
    
    e1 = e1_l - e1_r
    e2 = e2_l - e2_r
    
    return [e1, e2]
    
def get_ss_b(bvec_guess, p, f):
    n = p['n']
    F = lambda b: equations_ss_b(b, p, f)
    b_ss = fsolve(F, bvec_guess)
    c_ss = f['c'](b_ss, p, f)
    w_ss = f['w'](f['K'](b_ss), f['L'](n), p)
    r_ss = f['r'](f['K'](b_ss), f['L'](n), p)
    
    return b_ss, c_ss, w_ss, r_ss




In [16]:
def feasible(bvec_guess, f_params, functions):
    b_ss, c_ss, w_ss, r_ss = get_ss_b(bvec_guess, f_params, functions)
    print('b_ss: ', b_ss)
    K_cnstr = np.array([False])
    K_cnstr[0] = True if functions['K'](b_ss) <= 0 else False
    
    c_cnstr = c_ss
    c_cnstr = c_cnstr < 0 
        
    b_cnstr = np.array([False]*2)
    b_cnstr[0] = True if c_cnstr[0] or (c_cnstr[1]) else False
    b_cnstr[1] = True if c_cnstr[2] or (c_cnstr[1]) else False
    
    print('b_cnstr: ', b_cnstr)
    print('c_cnstr: ', c_cnstr)
    print('K_cnstr: ', K_cnstr)
    return b_cnstr, c_cnstr, K_cnstr

In [17]:
functions = {'L':get_L, 'K':get_K, 'r':get_r, 'u':get_u, 'u_p':u_prime, 
             'w':get_w,'c':get_c}

params = {'beta':0.96**(20), 'delta': 1- (1-0.05)**20, 'sigma':3 , \
          'A':1, 'alpha':0.35, 'n':np.array([1, 1, 0.2])}

In [20]:
parta = feasible(np.array([1, 1.2]), params, functions)

b_ss:  [7.57400704e+45 1.62947055e+45]
b_cnstr:  [ True False]
c_cnstr:  [ True False False]
K_cnstr:  [False]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The number of calls to function has reached maxfev = 600.
  warnings.warn(msg, RuntimeWarning)


In [21]:
partb = feasible(np.array([0.06, -0.001]), params, functions)

b_ss:  [ 0.06  -0.001]
b_cnstr:  [False False]
c_cnstr:  [False False False]
K_cnstr:  [False]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [22]:
partc = feasible(np.array([0.1, 0.1]), params, functions)

b_ss:  [0.01931274 0.05841159]
b_cnstr:  [False False]
c_cnstr:  [False False False]
K_cnstr:  [False]
